In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import LineString, Point, shape
import geopandas as gpd
from geopy.distance import great_circle
import xarray as xr

# From the ipynb driver, user input: ###################################
# User Inputs:
start_time_str       = '2023-01-01T00:00:00Z'
stop_time_str        = '2023-12-31T23:59:59Z'
query_limit          = 15e4
send_notification    = True
make_plot            = True
output_dir           = "/scratch/omg28/Data/"

# Convert start and stop times to datetime objects
start_time_simple = pd.to_datetime(start_time_str).strftime("%Y-%m-%d")
stop_time_simple = pd.to_datetime(stop_time_str).strftime("%Y-%m-%d")
analysis_year = pd.to_datetime(start_time_str).year

# Define grid
lat_bins = np.arange(-90, 90.1, 0.5)
lon_bins = np.arange(-180, 180.1, 0.5)
alt_bins_ft = np.arange(0, 55001, 1000)
alt_bins_m = alt_bins_ft * 0.3048
nlat, nlon, nalt = len(lat_bins)-1, len(lon_bins)-1, len(alt_bins_m)-1
########################################################################

# Define conflict countries and buffer degrees

conflict_countries = ['Russia', 'Ukraine', 'Libya', 'Syria', 'Sudan', 'Yemen']
buffer_degrees = 1.0

# Load country boundaries
world = gpd.read_file('ne_110m_admin_0_countries.zip')
name_col = 'NAME' if 'NAME' in world.columns else 'name'

# Get geometries for conflict countries
conflict_geometries = []
for c in conflict_countries:
    country_geom = world[world[name_col] == c].geometry
    if not country_geom.empty:
        conflict_geometries.append(country_geom.union_all().buffer(buffer_degrees))

# Create a union of `all conflict areas
if conflict_geometries:
    conflict_areas = conflict_geometries[0]
    for geom in conflict_geometries[1:]:
        conflict_areas = conflict_areas.union(geom)
    conflict_areas_buffered = conflict_areas.buffer(buffer_degrees)
else:
    # If no valid countries found, create empty geometry
    conflict_areas_buffered = Point(0, 0).buffer(0)


#TEST CODE ##################################
flights = pd.read_pickle('/scratch/omg28/Data/2023-01-01_to_2023-01-31_labeled.pkl')
test_flight = flight = flights.iloc[0]
era5_file = f"/scratch/omg28/Data/era5_wind_{analysis_year}.nc"
ds_era5 = xr.open_dataset(era5_file)
print(ds_era5)

#################################################################

# MATH HELPER FUNCTIONS ##################################
def secd(theta):
    """Convert degrees to seconds."""
    if theta%360 == 90 or theta%360 == 270:
        return np.nan
    return 1/np.cos(np.radians(theta))
    


1.0223405948650293


In [8]:
flights = pd.read_pickle('/scratch/omg28/Data/2023-01-01_to_2023-01-31_labeled.pkl')
flights.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1132523 entries, 0 to 1132522
Data columns (total 14 columns):
 #   Column               Non-Null Count    Dtype         
---  ------               --------------    -----         
 0   icao24               1132523 non-null  object        
 1   estdepartureairport  1132523 non-null  object        
 2   estarrivalairport    1132523 non-null  object        
 3   day                  1132523 non-null  int64[pyarrow]
 4   typecode             1132523 non-null  object        
 5   estdeparturelat      1132523 non-null  float64       
 6   estdeparturelong     1132523 non-null  float64       
 7   estdeparturealt_ft   1131831 non-null  float64       
 8   estarrivallat        1132523 non-null  float64       
 9   estarrivallong       1132523 non-null  float64       
 10  estarrivalalt_ft     1130594 non-null  float64       
 11  gc_km                1132523 non-null  float64       
 12  gc_FEAT_km           1132523 non-null  float64       
 1